Outlier detection   

In [98]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# === CONFIG ===
TARGET = 'Production'
ID_COL = 'PROP_ID'
NAME_COL = 'PROP_NAME'
START_COL = 'START_UP_YR'
CLOSE_COL = 'PROJ_CLOSURE_YR'
YEAR_COL = 'Year'
SAVE_DIR = 'Visualizations/Commodity Production/'

OUTLIER_THRESHOLD = 1.2
os.makedirs(SAVE_DIR, exist_ok=True)

# === FUNCTIONS ===
def filter_by_lifetime(df):
    return df[(df[YEAR_COL] >= df[START_COL]) & (df[YEAR_COL] <= df[CLOSE_COL])]

def dynamic_window(series, min_window=3, max_window=10, fraction=0.4):
    length = len(series)
    return max(min_window, min(int(length * fraction), max_window))

def detect_and_replace_outliers(series, threshold=1.2):
    window = dynamic_window(series)
    rolling_mean = series.rolling(window=window, center=True, min_periods=1).mean()
    rolling_std = series.rolling(window=window, center=True, min_periods=1).std()
    outliers = (np.abs(series - rolling_mean) > threshold * rolling_std)
    cleaned = series.copy()
    cleaned[outliers] = rolling_mean[outliers]
    return cleaned, outliers

def generate_full_lifetime_heatmap(df):
    mine_lifetimes = df[[ID_COL, START_COL, CLOSE_COL]].drop_duplicates()
    mine_lifetimes['Year_range'] = mine_lifetimes.apply(
        lambda row: list(range(row[START_COL], row[CLOSE_COL] + 1)), axis=1
    )
    df_lifetime_full = mine_lifetimes.explode('Year_range').rename(columns={'Year_range': 'Year'})
    df_lifetime_full['Year'] = df_lifetime_full['Year'].astype(int)

    df_renamed = df.rename(columns={YEAR_COL: 'Prod_Year'})
    df_expanded = pd.merge(
        df_lifetime_full,
        df_renamed[[ID_COL, 'Prod_Year', 'Production_Original']],
        left_on=[ID_COL, 'Year'],
        right_on=[ID_COL, 'Prod_Year'],
        how='left'
    ).drop(columns=['Prod_Year'])

    df_expanded['Missing'] = df_expanded['Production_Original'].isna().astype(int)
    df_pivot = df_expanded.pivot_table(index=ID_COL, columns='Year', values='Missing', fill_value=-1)
    df_melt = df_pivot.reset_index().melt(id_vars=ID_COL, var_name='Year', value_name='Status')
    df_melt['Category'] = df_melt['Status'].map({-1: 'Not Applicable', 0: 'Value Present', 1: 'Missing'})

    sample_ids_heatmap = df_melt[ID_COL].unique()[:10]
    df_heatmap_pivot = df_melt[df_melt[ID_COL].isin(sample_ids_heatmap)].pivot(
        index=ID_COL, columns='Year', values='Category'
    )
    category_to_num = {'Not Applicable': 0, 'Missing': 1, 'Value Present': 2}
    heatmap_matrix = df_heatmap_pivot.replace(category_to_num)

    plt.figure(figsize=(16, 6))
    sns.heatmap(
        heatmap_matrix,
        cmap=sns.color_palette(["white", "#c51b7d", "#4d9221"]),
        cbar_kws={'ticks': [0.5, 1.5, 2.5], 'format': '%d'},
        linewidths=0.1,
        linecolor='gray'
    )
    plt.title("Heatmap of Missing Data Categories (Full Mine Lifetime, Sample Mines)")
    plt.xlabel("Year")
    plt.ylabel("Mine ID")
    plt.tight_layout()
    plt.savefig(os.path.join(SAVE_DIR, "heatmap_missing_values_full_lifetime.png"))
    plt.close()

def generate_completeness_summary(df_full_lifetime, outlier_counts):
    mine_lifetimes = df_full_lifetime[[ID_COL, START_COL, CLOSE_COL]].drop_duplicates()
    mine_lifetimes['Year_range'] = mine_lifetimes.apply(
        lambda row: list(range(row[START_COL], row[CLOSE_COL] + 1)), axis=1
    )
    df_lifetime_full = mine_lifetimes.explode('Year_range').rename(columns={'Year_range': 'Year'})
    df_lifetime_full['Year'] = df_lifetime_full['Year'].astype(int)

    df_renamed = df_full_lifetime.rename(columns={YEAR_COL: 'Prod_Year'})
    df_expanded = pd.merge(
        df_lifetime_full,
        df_renamed[[ID_COL, 'Prod_Year', 'Production_Original']],
        left_on=[ID_COL, 'Year'],
        right_on=[ID_COL, 'Prod_Year'],
        how='left'
    ).drop(columns=['Prod_Year'])

    completeness = (
        df_expanded.groupby(ID_COL)['Production_Original']
        .apply(lambda x: x.notna().sum() / len(x))
        .reset_index(name='Completeness')
    )
    completeness = completeness.merge(outlier_counts, on=ID_COL, how='left')

    plt.figure(figsize=(10, 6))
    sns.histplot(completeness['Completeness'], bins=30, kde=False, color="steelblue")
    plt.axvline(0.25, color="red", linestyle="--")
    plt.axvline(0.5, color="red", linestyle="--")
    plt.axvline(0.75, color="red", linestyle="--")
    plt.title("Histogram of Production Data Completeness per Mine")
    plt.xlabel("Proportion of Non-Missing Data")
    plt.ylabel("Number of Mines")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(SAVE_DIR, "histplot_completeness.png"))
    plt.close()

    completeness.to_excel("Data Output/Commodity Production/Completeness_Summary.xlsx", index=False)

def plot_timeseries_with_outliers(df, id_list, original_col='Production_Original', cleaned_col='Production'):
    sample = df[df[ID_COL].isin(id_list)]
    g = sample.groupby(ID_COL)
    n = len(id_list)
    ncols = 4
    nrows = int(np.ceil(n / ncols))
    fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols * 5, nrows * 4), sharex=False, sharey=False)
    axs = axs.flatten()

    for i, (mine_id, group) in enumerate(g):
        ax = axs[i]
        ax.plot(group[YEAR_COL], group[original_col], label="Original", marker='o', linestyle='-')
        ax.plot(group[YEAR_COL], group[cleaned_col], label="Cleaned", marker='x', linestyle='--')
        ax.set_title(f"Mine {mine_id}")
        ax.set_xlabel("Year")
        ax.set_ylabel("Production")
        ax.legend()

    for j in range(i+1, len(axs)):
        fig.delaxes(axs[j])

    fig.tight_layout()
    fig.savefig(os.path.join(SAVE_DIR, "timeseries_outliers_grid.png"))
    plt.close()

def main():
    path = 'Data Output/Commodity Production/Commodity_Production-1980_2023_long.xlsx'
    df = pd.read_excel(path)

    # Rename original Production column to Production_Original at the start
    df = df.rename(columns={'Production': 'Production_Original'})
    
    # Update TARGET to use the original column for filtering and processing
    TARGET_ORIGINAL = 'Production_Original'

    # Filter by mine lifetime
    df_filtered = filter_by_lifetime(df)

    # Clean and detect outliers using dynamic window
    df_outliers = df_filtered[(df_filtered[TARGET_ORIGINAL].notna()) & (df_filtered[TARGET_ORIGINAL] > 0)]
    df_outliers = df_outliers.groupby(ID_COL).filter(lambda x: (x[TARGET_ORIGINAL].notna() & (x[TARGET_ORIGINAL] > 0)).sum() >= 5)
    df_cleaned = df_outliers.copy()
    df_cleaned = df_cleaned.sort_values([ID_COL, YEAR_COL])

    cleaned_list = []
    outlier_list = []

    for mine_id, group in df_cleaned.groupby(ID_COL):
        series = group[TARGET_ORIGINAL]
        cleaned, outliers = detect_and_replace_outliers(series, threshold=OUTLIER_THRESHOLD)
        cleaned_list.extend(cleaned)
        outlier_list.extend(outliers)

    # Create the new Production column with cleaned data
    df_cleaned['Production'] = cleaned_list
    df_cleaned['Outlier'] = outlier_list

    # Save cleaned file
    df_cleaned.to_excel("Data Output/Commodity Production/Commodity_Production-1980_2023_long_final.xlsx", index=False)

    # Generate outlier summary
    outlier_counts = df_cleaned.groupby(ID_COL)['Outlier'].sum().reset_index(name='Num_Outliers')

    # Visualizations
    sample_ids = df_cleaned[ID_COL].unique()[:20]
    plot_timeseries_with_outliers(df_cleaned, sample_ids)
    generate_full_lifetime_heatmap(df)
    generate_completeness_summary(df, outlier_counts)

if __name__ == '__main__':
    main()


C:\Users\leshe\AppData\Local\Temp\ipykernel_21740\1718636777.py:63: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


Hubbert and Femp imputation 

In [99]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from scipy import stats
from tqdm import tqdm
from joblib import Parallel, delayed

############################################################Parameters############################################################
# Changed to only include 'production' as the target variable
targets = ['Production']

lower_bounds = {
    'hubbert': (0, 0, 0),  # log-transformed L and k
    'femp': (0, 0)  # log-transformed R0, logit-transformed C
}

upper_bounds = {
    'hubbert': (np.inf, np.inf, np.inf),
    'femp': (np.inf, 1)  # No upper bound needed for log-transformed variables
}
min_sample_size = 5

np.random.seed(42)

############################################################Functions############################################################
def safe_exp(x):
    try:
        return np.exp(x)
    except OverflowError:
        return np.inf  # Or a reasonable max value
    
def hubbert_transformed(t, log_L, log_k, t0):
    """
    Hubbert model with log-transformed parameters.
    
    Parameters:
        t (np.array): Time
        log_L (float): Log of maximum production
        log_k (float): Log of growth rate
        t0 (float): Time of peak production
    
    Returns:
        np.array: Production at time
    """
    L = safe_exp(log_L)  # Ensure L is strictly positive
    k = safe_exp(log_k)  # Ensure k is strictly positive
    return L / (1 + np.exp(-k * (t - t0)))

def femp_transformed(t, log_R0, C_transformed):
    """
    FEMP model with log and logit-transformed parameters.
    
    Parameters:
        t (np.array): Time
        log_R0 (float): Log of initial reserves
        C_transformed (float): Logit of production-to-reserve ratio
    
    Returns:
        np.array: Production at time
    """
    R0 = safe_exp(log_R0)  # Ensure R0 is strictly positive
    C = 1 / (1 + safe_exp(-C_transformed))  # Ensure C stays between 0 and 1
    return R0 * (1 - (1 - C)**t)

def hubbert_deriv_transformed(t, log_L, log_k, t0):
    """
    Hubbert model derivative with log-transformed parameters.
    """
    L = safe_exp(log_L)  # Ensure L is strictly positive
    k = safe_exp(log_k)  # Ensure k is strictly positive
    return L * k * np.exp(-k * (t - t0)) / (1 + np.exp(-k * (t - t0)))**2

def femp_deriv_transformed(t, log_R0, C_transformed):
    """
    FEMP model derivative with log and logit-transformed parameters.
    """
    R0 = safe_exp(log_R0)  # Ensure R0 is strictly positive
    C = 1 / (1 + safe_exp(-C_transformed))  # Ensure C stays between 0 and 1
    return -R0 * np.log(1 - C) * (1 - C)**t

def hubbert_model(t, L, k, t0):
    '''
    Hubbert model for production
    
    Parameters:
        t (np.array): Time
        L (float): Maximum production
        k (float): Growth rate
        t0 (float): Time of peak production
        
    Returns:
        np.array: Production at time
    
    '''
    return L / (1 + np.exp(-k * (t - t0)))

def femp(t, R0, C):
    '''
    Exponential model for production
    
    Parameters:
        t (np.array): Time
        R0 (float): Initial reserves
        C (float): Production to reserve ratio
        
    Returns:
        np.array: Production at time
    
    '''
    return R0 * (1-(1-C)**t)

def femp_deriv(t, R0, C):
    '''
    Derivative of the exponential model for production
    
    Parameters:
        t (np.array): Time
        R0 (float): Initial reserves
        C (float): Production to reserve ratio
        
    Returns:
        np.array: Production at time
    
    '''
    return - R0 * np.log(1-C) * (1-C)**t

def hubbert_deriv(t, L, k, t0):
    '''
    Derivative of the Hubbert model for production
    
    Parameters:
        t (np.array): Time
        L (float): Maximum production
        k (float): Growth rate
        t0 (float): Time of peak production
        
    Returns:
        np.array: Production at time
    
    '''
    return L * k * safe_exp(-k * (t - t0)) / (1 + safe_exp(-k * (t - t0)))**2

def prep_init_guesses(model_name, sample, t):
    if model_name == 'hubbert':
        L_guess = sample[t].cumsum().max()  
        k_guess = sample[t].mean() / L_guess  
        t0_guess = sample.loc[sample[t].idxmax(), 'Year_diff'].astype(int) 
        return (L_guess, k_guess, t0_guess)

    elif model_name == 'femp':
        R0_guess = sample[t].cumsum().max()
        C_guess = sample[t].mean() / R0_guess
        return (R0_guess, C_guess)

def fit_prod_model(mine, prod_data, models, targets, lower_bounds, upper_bounds, min_sample_size):
    results = []
    data_records = []

    for t in targets:
        sample = prod_data.dropna(subset=t)

        sample_size = len(sample)
        if sample_size < min_sample_size:
            continue

        # Get the projected closure year and startup year for this mine
        proj_closure_yr = sample['PROJ_CLOSURE_YR'].iloc[0]
        start_up_yr = sample['START_UP_YR'].iloc[0]
        
        # Calculate mine lifetime using closure year
        mine_lifetime = int(proj_closure_yr - start_up_yr)

        for model_name, model in models.items():
            years = sample['Year_diff']
            init_guess = prep_init_guesses(model_name, sample, t)

            try:
                # Fit model with transformed parameters
                popt, pcov = curve_fit(
                    model,
                    years.astype(int),
                    sample[t],
                    p0=init_guess,
                    maxfev=10000,
                    bounds=(lower_bounds[model_name], upper_bounds[model_name])
                )

                perr = np.sqrt(np.diag(pcov))  # Standard errors

                # Transform parameters back to original scale
                if model_name == 'hubbert':
                    L_est, k_est, t0_est = popt[0], popt[1], popt[2]
                    L_err, k_err, t0_err = perr[0], perr[1], perr[2]

                else:  # FEMP model
                    R0_est, C_est = popt[0], popt[1]
                    R0_err, C_err = perr[0], perr[1]

                # Predictions on actual data points
                pred = model(years.astype(int), *popt)

                # p-values
                t_stats = popt / perr
                p_values = [2 * (1 - stats.t.cdf(np.abs(t), sample_size - len(popt))) for t in t_stats]
                
                r2 = np.corrcoef(sample[t], pred)[0, 1] ** 2
                rmse = np.sqrt(np.mean((sample[t] - pred) ** 2))
                nrmse = rmse / sample[t].max()

                # Store results
                results.append({
                    'PROP_ID': mine,
                    'Target_var': t,
                    'Model': model_name,
                    'START_UP_YR': start_up_yr,
                    'PROJ_CLOSURE_YR': proj_closure_yr,
                    'Mine_Lifetime': mine_lifetime,
                    'R2': r2,
                    'RMSE': rmse,
                    'NRMSE': nrmse,
                    'P1_value': L_est if model_name == 'hubbert' else R0_est,
                    'P2_value': k_est if model_name == 'hubbert' else C_est,
                    'P3_value': t0_est if model_name == 'hubbert' else np.nan,
                    'P1_err': L_err if model_name == 'hubbert' else R0_err,
                    'P2_err': k_err if model_name == 'hubbert' else C_err,
                    'P3_err': t0_err if model_name == 'hubbert' else np.nan,
                    'P1_pval': p_values[0],
                    'P2_pval': p_values[1],
                    'P3_pval': p_values[2] if model_name == 'hubbert' else np.nan
                })

                # Collect actual observed data and predictions
                obs_data_records = pd.DataFrame({
                    'PROP_ID': mine,
                    'Year_diff': years,
                    'Year': years + start_up_yr,
                    'Target_var': t,
                    'START_UP_YR': start_up_yr,
                    'PROJ_CLOSURE_YR': proj_closure_yr,
                    'Observed': sample[t],
                    'Predicted': pred,
                    'Residual': sample[t] - pred,
                    'Model': model_name,
                    'Data_Type': 'Observed'
                })
                
                # Add predictions up to closure year
                # Create prediction data for all years up to closure
                full_years = np.arange(0, mine_lifetime + 1)
                
                # Filter out years that are already in the observed data
                future_years = np.setdiff1d(full_years, years)
                
                if len(future_years) > 0:
                    future_pred = model(future_years, *popt)
                    
                    # Create dataframe for future predictions
                    future_data_records = pd.DataFrame({
                        'PROP_ID': mine,
                        'Year_diff': future_years,
                        'Year': future_years + start_up_yr,
                        'Target_var': t,
                        'START_UP_YR': start_up_yr,
                        'PROJ_CLOSURE_YR': proj_closure_yr,
                        'Observed': np.nan,
                        'Predicted': future_pred,
                        'Residual': np.nan,
                        'Model': model_name,
                        'Data_Type': 'Predicted_Future'
                    })
                    
                    # Combine observed and future predictions
                    all_data_records = pd.concat([obs_data_records, future_data_records])
                else:
                    all_data_records = obs_data_records
                
                data_records.append(all_data_records)

            except RuntimeError as e:
                # Handle fitting failures
                print(f"Fitting error for mine {mine}, target {t}, model {model_name}: {e}")
                results.append({
                    'PROP_ID': mine,
                    'Target_var': t,
                    'Model': model_name,
                    'START_UP_YR': start_up_yr,
                    'PROJ_CLOSURE_YR': proj_closure_yr,
                    'Mine_Lifetime': mine_lifetime,
                    'R2': np.nan,
                    'RMSE': np.nan,
                    'NRMSE': np.nan,
                    'P1_value': np.nan,
                    'P2_value': np.nan,
                    'P3_value': np.nan,
                    'P1_err': np.nan,
                    'P2_err': np.nan,
                    'P3_err': np.nan,
                    'P1_pval': np.nan,
                    'P2_pval': np.nan,
                    'P3_pval': np.nan
                })

    return results, data_records

def prep_data(file_path):
    """
    Prepare data for modeling
    
    Parameters:
        file_path (str): Path to the production data Excel file
        
    Returns:
        pd.DataFrame: Prepared production data
    """
    prod_data = pd.read_excel(file_path)
    
    # Calculate year difference
    if 'Year' in prod_data.columns:
        prod_data['Year_diff'] = prod_data['Year'] - prod_data['START_UP_YR']
    else:
        # If no Year column, assuming sequential years are provided
        prod_data['Year_diff'] = prod_data.groupby('PROP_ID').cumcount()
    
    assert prod_data['Year_diff'].min() >= 0, 'Negative year difference'
    
    return prod_data

############################################################Main############################################################
def main_fitting_loop(data_path, output_dir='Data Output/Cumulative Production/'):
    """
    Fit models to production data
    
    Parameters:
        data_path (str): Path to the production data Excel file
        output_dir (str): Directory to save results
        
    Returns:
        pd.DataFrame: DataFrame of model fits
    """
    import os
    os.makedirs(output_dir, exist_ok=True)
    
    # Use derivative models for fitting
    models = {'hubbert': hubbert_deriv, 'femp': femp_deriv}
    prod_data = prep_data(data_path)
    
    p_group = prod_data.groupby('PROP_ID')

    # Parallelize over mines
    results_data = Parallel(n_jobs=-1)(delayed(fit_prod_model)(
        mine, group, models, targets, lower_bounds, upper_bounds, min_sample_size
    ) for mine, group in tqdm(p_group, desc='Fitting models to production data'))

    # Unpack parallelized results
    all_results, all_data_records = zip(*results_data)

    # Combine results if not empty
    res_df = pd.concat([pd.DataFrame(res) for res in all_results if res])
    
    # Properly handle data records - flatten the list of dataframes
    flattened_data_records = []
    for sublist in all_data_records:
        if isinstance(sublist, list):
            for item in sublist:
                if isinstance(item, pd.DataFrame) and not item.empty:
                    flattened_data_records.append(item)
        elif isinstance(sublist, pd.DataFrame) and not sublist.empty:
            flattened_data_records.append(sublist)
    
    data_records = pd.concat(flattened_data_records) if flattened_data_records else pd.DataFrame()
    
    # Save results as Excel files
    res_df.to_excel(f'{output_dir}/production_model_fits_lifetime.xlsx', index=False)
    data_records.to_excel(f'{output_dir}/data_records_lifetime.xlsx', index=False)
    
    
    return res_df


if __name__ == '__main__':
    # Use the specified input file path
    data_path = "Data Output/Commodity Production/Commodity_Production-1980_2023_long_final.xlsx"
    main_fitting_loop(data_path)

Fitting models to production data: 100%|██████████| 723/723 [01:17<00:00,  9.30it/s]


In [100]:
import pandas as pd
import numpy as np
import os

def clean_empty_p_values():
    """
    Deletes all PROP_IDs from both production_model_fits_lifetime.xlsx and
    data_records_lifetime.xlsx if they have any empty p-values.
    Prints summary and saves cleaned files back to their original paths.
    """
    fits_path = 'Data Output/Cumulative Production/production_model_fits_lifetime.xlsx'
    records_path = 'Data Output/Cumulative Production/data_records_lifetime.xlsx'

    # Check both files exist
    for path in [fits_path, records_path]:
        if not os.path.exists(path):
            print(f"File not found: {path}")
            return

    # Load fits data
    print(f"Loading data from {fits_path}")
    df_fits = pd.read_excel(fits_path)

    # Check required columns
    required_cols = ['PROP_ID', 'P1_value', 'P2_value']
    if not all(col in df_fits.columns for col in required_cols):
        print(f"Missing required columns in {fits_path}")
        return

    # Initial stats
    total_mines = df_fits['PROP_ID'].nunique()
    total_rows = len(df_fits)
    print(f"Initial: {total_rows} rows, {total_mines} unique mines")

    # Identify mines with empty P values
    mask_p1 = df_fits['P1_value'].isna()
    mask_p2 = df_fits['P2_value'].isna()
    mask_any_empty = mask_p1 | mask_p2
    mines_to_drop = df_fits.loc[mask_any_empty, 'PROP_ID'].unique()

    if len(mines_to_drop) == 0:
        print("No missing p-values found.")
        return

    # Print details
    print(f"\nDropping {len(mines_to_drop)} mines with missing p-values:")
    for mine_id in mines_to_drop:
        missing = []
        if df_fits.loc[(df_fits['PROP_ID'] == mine_id) & mask_p1].any().any():
            missing.append("P1")
        if df_fits.loc[(df_fits['PROP_ID'] == mine_id) & mask_p2].any().any():
            missing.append("P2")
        print(f"  - PROP_ID {mine_id}: missing {', '.join(missing)}")

    # Filter both datasets
    df_fits_clean = df_fits[~df_fits['PROP_ID'].isin(mines_to_drop)]

    print(f"\nLoading corresponding records from {records_path}")
    df_records = pd.read_excel(records_path)
    if 'PROP_ID' not in df_records.columns:
        print(f"Missing PROP_ID column in {records_path}")
        return

    df_records_clean = df_records[~df_records['PROP_ID'].isin(mines_to_drop)]

    # Save cleaned datasets
    df_fits_clean.to_excel(fits_path, index=False)
    df_records_clean.to_excel(records_path, index=False)

    # Summary
    print(f"\nCleaned data saved:")
    print(f"  - {len(df_fits) - len(df_fits_clean)} rows removed from fits file")
    print(f"  - {len(df_records) - len(df_records_clean)} rows removed from records file")
    print(f"  - {len(mines_to_drop)} unique mines removed")

if __name__ == "__main__":
    clean_empty_p_values()


Loading data from Data Output/Cumulative Production/production_model_fits_lifetime.xlsx
Initial: 1446 rows, 723 unique mines
No missing p-values found.


Results Analysis

In [101]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from scipy.stats import skew, kurtosis
import os

# Load your data
modelres = pd.read_excel("Data Output/Cumulative Production/production_model_fits_lifetime.xlsx")
data_records = pd.read_excel("Data Output/Cumulative Production/data_records_lifetime.xlsx")

# Constants
sig = 0.05

# ------- Classification Function --------
def classify_model_results(df):
    df['femp_significant'] = (df['Model'] == 'femp') & (df[['P1_pval', 'P2_pval']] < sig).all(axis=1)
    df['hubbert_significant'] = (df['Model'] == 'hubbert') & (df[['P1_pval', 'P2_pval', 'P3_pval']] < sig).all(axis=1)

    agg = df.groupby(['PROP_ID']).agg({
        'femp_significant': 'any',
        'hubbert_significant': 'any',
        'RMSE': lambda x: x.iloc[1] - x.iloc[0] if len(x) == 2 else np.nan,
        'R2': lambda x: x.iloc[1] - x.iloc[0] if len(x) == 2 else np.nan
    }).reset_index()

    def choose_class(row):
        if row['femp_significant'] and row['hubbert_significant']:
            if row['RMSE'] < 0 and row['R2'] > 0:
                return 'F'
            elif row['RMSE'] > 0 and row['R2'] < 0:
                return 'H'
            else:
                return 'H'
        elif row['femp_significant']:
            return 'F'
        elif row['hubbert_significant']:
            return 'H'
        else:
            return 'N'

    agg['Class'] = agg.apply(choose_class, axis=1)
    return df.merge(agg[['PROP_ID', 'Class']], on='PROP_ID')

# ------- Summary Statistics --------
def summarize_fit_stats(df):
    summary = df.groupby(['Model'])[['R2', 'RMSE', 'NRMSE']].agg(['mean', 'std', 'median', skew, kurtosis]).round(2)
    summary.to_excel("Data Output/Cumulative Production/production_model_summary.xlsx")
    return summary

# ------- Matplotlib Plots --------
def plot_histograms(df):
    for metric in ['R2', 'RMSE', 'NRMSE']:
        plt.figure(figsize=(8, 6))
        for model in df['Model'].unique():
            subset = df[df['Model'] == model]
            values = subset[metric] / 1e6 if metric == 'RMSE' else subset[metric]
            plt.hist(values, bins=20, alpha=0.6, label=model)
        plt.title(f"Histogram of {metric}")
        plt.xlabel(metric + (" (Mt)" if metric == "RMSE" else ""))
        plt.ylabel("Frequency")
        plt.legend()
        plt.tight_layout()
        plt.savefig(f"Visualizations/Cumulative Production/{metric}_histogram.png")
        plt.close()

def plot_class_bar(df):
    class_counts = df['Class'].value_counts(normalize=True) * 100
    class_counts.plot(kind='bar', color='skyblue')
    plt.title("Model Classification Share")
    plt.ylabel("Percentage")
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.savefig("Visualizations/Cumulative Production/class_bar_chart.png")
    plt.close()

def plot_error_scatter(df):
    plt.figure(figsize=(10, 8))
    for model in df['Model'].unique():
        subset = df[df['Model'] == model]
        plt.scatter(subset['Observed'], subset['Predicted'], alpha=0.3, label=model)
    plt.xscale('log')
    plt.yscale('log')
    plt.plot([1, 1e8], [1, 1e8], 'k--')
    plt.xlabel("Observed")
    plt.ylabel("Predicted")
    plt.title("Observed vs Predicted (log-log)")
    plt.legend()
    plt.tight_layout()
    plt.savefig("Visualizations/Cumulative Production/error_scatter.png")
    plt.close()

def plot_error_time_series(df):
    df['Error'] = df['Predicted'] - df['Observed']
    df = df.dropna(subset=['Error'])
    df['Error'] = StandardScaler().fit_transform(df['Error'].values.reshape(-1, 1))
    df = df[df['Year'] > 1950]
    plt.figure(figsize=(12, 6))
    for model in df['Model'].unique():
        subset = df[df['Model'] == model]
        plt.plot(subset['Year'], subset['Error'], '.', alpha=0.3, label=model)
    plt.axhline(0, color='black', linestyle='--')
    plt.xlabel("Year")
    plt.ylabel("Standardized Error")
    plt.title("Error Over Time")
    plt.legend()
    plt.tight_layout()
    plt.savefig("Visualizations/Cumulative Production/error_time_series.png")
    plt.close()

def plot_boxplot_fit_metrics(df, metric):
    plt.figure(figsize=(8, 6))
    df.boxplot(column=metric, by='Model')
    plt.title(f"Boxplot of {metric} by Model")
    plt.suptitle("")
    plt.xlabel("Model")
    plt.ylabel(metric)
    plt.tight_layout()
    plt.savefig(f"Visualizations/Cumulative Production/{metric}_boxplot.png")
    plt.close()

# ------- Main Execution --------
if __name__ == '__main__':
    modelres = classify_model_results(modelres)
    print(summarize_fit_stats(modelres))

    data_records = data_records.merge(modelres[['PROP_ID', 'Model', 'Class']], on=['PROP_ID', 'Model'], how='left')

    plot_histograms(modelres)
    plot_class_bar(modelres)
    plot_error_scatter(data_records)
    plot_error_time_series(data_records)
    plot_boxplot_fit_metrics(modelres, 'R2')
    plot_boxplot_fit_metrics(modelres, 'RMSE')


           R2                                   RMSE                        \
         mean   std median  skew kurtosis       mean         std    median   
Model                                                                        
femp     0.44  0.30   0.45  0.02    -1.36  706087.11  2828305.11  25844.73   
hubbert  0.62  0.28   0.72 -0.76    -0.61  426145.47  3075961.52  13487.98   

                        NRMSE                              
          skew kurtosis  mean   std median  skew kurtosis  
Model                                                      
femp     10.31   154.17  0.24  0.13   0.22  1.10     1.53  
hubbert  19.59   448.06  0.12  0.07   0.11  1.91     5.55  


C:\Users\leshe\AppData\Local\Temp\ipykernel_21740\349211397.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

Cumulative Production calculation

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

# ----------------------------
# Config
# ----------------------------
INPUT_FILE = "Data Output/Cumulative Production/production_model_fits_lifetime.xlsx"
OUTPUT_FILE = "Data Output/Cumulative Production/Cumulative_Production.xlsx"

# ----------------------------
# Models
# ----------------------------
def hubbert_model(t, L, k, t0):
    return L / (1 + np.exp(-k * (t - t0)))

def femp(t, R0, C):
    return R0 * (1 - (1 - C) ** t)

# ----------------------------
# Load data
# ----------------------------
df = pd.read_excel(INPUT_FILE)
df["Model"] = df["Model"].str.lower()

# ----------------------------
# Calculate cumulative production
# ----------------------------
results = []

for _, row in df.iterrows():
    t = row["Mine_Lifetime"]

    if row["Model"] == "hubbert":
        L, k, t0 = row["P1_value"], row["P2_value"], row["P3_value"]
        cum_prod = hubbert_model(t, L, k, t0)

    elif row["Model"] == "femp":
        R0, C = row["P1_value"], row["P2_value"]
        cum_prod = femp(t, R0, C)

    else:
        continue

    results.append([row["PROP_ID"], row["Model"], t, cum_prod])

# ----------------------------
# Save results
# ----------------------------
out = pd.DataFrame(results, columns=["PROP_ID", "Model", "Mine_Lifetime", "Cumulative_Production"])
Path(OUTPUT_FILE).parent.mkdir(parents=True, exist_ok=True)
out.to_excel(OUTPUT_FILE, index=False)

print(f"Saved results to: {OUTPUT_FILE}")


Saved results to: Data Output/Cumulative Production/Cumulative_Production.xlsx


: 

In [6]:
import pandas as pd

# File paths
cumprod_path = "Data Output/Cumulative Production/Cumulative_Production.xlsx"
commodity_path = "Data Output/Commodity Production/Commodity_Production-1980_2023_final.xlsx"
output_path = "Data Output/Commodity Production/Commodity_Production-1980_2023_final.xlsx"

# Load cumulative production data
cumprod_df = pd.read_excel(cumprod_path)

# Keep only rows for Hubbert model
hubbert_df = cumprod_df[cumprod_df["Model"] == "hubbert"]

# Keep only the relevant columns and rename
hubbert_df = hubbert_df[["PROP_ID", "Cumulative_Production"]].rename(
    columns={"Cumulative_Production": "CumProd_Tonnes"}
)

# Load commodity production data
commodity_df = pd.read_excel(commodity_path)

# Merge on PROP_ID
merged_df = commodity_df.merge(hubbert_df, on="PROP_ID", how="left")

# Save to a new file
merged_df.to_excel(output_path, index=False)

print(f"Merged file saved to: {output_path}")


KeyError: "['Cumulative_Production'] not in index"

Cumulative production vs reserves comparison

3) compare to reserves metal, except bauxite where you compare to reserves ore
4) drop mines that are >3x the reserves

In [104]:
#!/usr/bin/env python3
"""
Compare cumulative production with reserves and save results to Excel.

- Uses ONLY the commodity production file.
- Cumulative: CumProd_Tonnes
- Reserves (Bauxite): Reserves_Tonnage
- Reserves (all others): Reserves_Contained
"""

import pandas as pd
import numpy as np
import os

# =============================================================================
# CONFIGURATION
# =============================================================================
PRODUCTION_FILE = "Data Output/Commodity Production/Commodity_Production-1980_2023_final.xlsx"

OUTPUT_DIR = "Data Output/Cumulative Production/Comparison_Results/Reserves_Comparison"
os.makedirs(OUTPUT_DIR, exist_ok=True)

OUTPUT_FILE = os.path.join(OUTPUT_DIR, "cumulative_vs_reserves.xlsx")

# Column names
PROP_ID_COL = "PROP_ID"
PRIMARY_COMMODITY_COL = "PRIMARY_COMMODITY"
CUMULATIVE_COL = "CumProd_Tonnes"
RESERVES_CONTAINED_COL = "Reserves_Contained"
RESERVES_TONNAGE_COL = "Reserves_Tonnage"

# =============================================================================
# MAIN
# =============================================================================
def main():
    print("Running cumulative production vs reserves comparison (single file)...")

    # Load from the commodity production file
    df = pd.read_excel(PRODUCTION_FILE, engine="openpyxl")

    # Keep only needed columns and one row per PROP_ID
    keep_cols = [
        PROP_ID_COL,
        PRIMARY_COMMODITY_COL,
        CUMULATIVE_COL,
        RESERVES_CONTAINED_COL,
        RESERVES_TONNAGE_COL,
    ]
    df = df[keep_cols].groupby(PROP_ID_COL, as_index=False).agg("first")

    # Ensure numeric where needed
    for col in [CUMULATIVE_COL, RESERVES_CONTAINED_COL, RESERVES_TONNAGE_COL]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    # Choose reserves column based on commodity
    is_bauxite = df[PRIMARY_COMMODITY_COL].astype(str).str.strip().str.lower().eq("bauxite")
    df["Reserves_Compare"] = np.where(
        is_bauxite,
        df[RESERVES_TONNAGE_COL],
        df[RESERVES_CONTAINED_COL],
    )

    # Drop rows missing either cumulative or reserves
    df = df.dropna(subset=[CUMULATIVE_COL, "Reserves_Compare"])

    # Compute metrics
    df["Absolute_Difference"] = (df[CUMULATIVE_COL] - df["Reserves_Compare"]).abs()
    df["Raw_Difference"] = df[CUMULATIVE_COL] - df["Reserves_Compare"]
    df["Percent_Difference"] = (df["Raw_Difference"] / df["Reserves_Compare"]) * 100
    df["Cumulative_to_Reserves_Ratio"] = df[CUMULATIVE_COL] / df["Reserves_Compare"]

    # Save
    df.to_excel(OUTPUT_FILE, index=False)
    print(f"Saved results to {OUTPUT_FILE}")


if __name__ == "__main__":
    main()



Running cumulative production vs reserves comparison (single file)...
Saved results to Data Output/Cumulative Production/Comparison_Results/Reserves_Comparison\cumulative_vs_reserves.xlsx


In [105]:
"""
Simple script to remove mines with high Cumulative_to_Reserves_Ratio (>3.4) 
from the main dataset.
"""

import pandas as pd

# File paths
MAIN_DATASET_FILE = "Data Output/Commodity Production/Commodity_Production-1980_2023_final.xlsx"
COMPARISON_FILE = "Data Output/Cumulative Production/Comparison_Results/Reserves_Comparison/cumulative_vs_reserves.xlsx"

# Filtering criteria
RATIO_THRESHOLD = 3.4

# Column names
PROP_ID_COL = 'PROP_ID'
RATIO_COL = 'Cumulative_to_Reserves_Ratio'

def main():
    """Remove high-ratio mines from the main dataset."""
    
    print("Loading datasets...")
    
    # Load main dataset
    main_df = pd.read_excel(MAIN_DATASET_FILE, engine='openpyxl')
    print(f"Main dataset: {len(main_df):,} records")
    
    # Load comparison results (cumulative vs reserves)
    comp_df = pd.read_excel(COMPARISON_FILE, engine='openpyxl')
    
    # Ensure ratio is numeric
    comp_df[RATIO_COL] = pd.to_numeric(comp_df[RATIO_COL], errors='coerce')
    
    # Find mines with high ratios
    high_ratio_mines = comp_df[comp_df[RATIO_COL] > RATIO_THRESHOLD]
    prop_ids_to_remove = set(high_ratio_mines[PROP_ID_COL].dropna().unique())
    
    print(f"Mines to remove: {len(prop_ids_to_remove)}")
    print(f"PROP_IDs: {sorted(prop_ids_to_remove)}")
    
    # Filter main dataset
    filtered_df = main_df[~main_df[PROP_ID_COL].isin(prop_ids_to_remove)]
    
    # Update the main file
    filtered_df.to_excel(MAIN_DATASET_FILE, index=False)
    
    print(f"Updated dataset: {len(filtered_df):,} records")
    print(f"Removed: {len(main_df) - len(filtered_df):,} records")
    print("Done!")

if __name__ == "__main__":
    main()


Loading datasets...
Main dataset: 723 records
Mines to remove: 122
PROP_IDs: [24470, 24474, 24503, 24505, 24732, 24733, 24774, 24783, 24815, 25687, 25699, 25715, 25720, 25727, 25751, 25753, 25754, 25831, 26073, 26495, 26575, 26711, 26722, 27027, 27134, 27139, 27145, 27146, 27163, 27165, 27166, 27167, 27213, 27243, 27280, 27283, 27291, 27303, 27345, 27384, 27539, 27542, 27543, 27612, 27616, 27632, 27653, 27747, 27830, 27868, 27971, 27981, 28004, 28021, 28245, 28250, 28264, 28283, 28296, 28316, 28362, 28363, 28364, 28428, 28585, 28596, 28703, 28955, 29057, 29073, 29195, 29246, 29347, 29377, 29438, 29544, 29792, 29980, 30277, 30526, 30656, 31300, 31316, 31560, 31563, 31631, 31651, 31758, 31818, 31827, 31833, 31895, 31907, 31934, 32142, 32169, 32642, 32779, 32931, 35185, 35378, 35755, 35780, 36464, 36896, 37326, 38727, 54759, 56047, 57368, 57500, 59345, 64042, 66063, 66682, 66683, 68175, 74744, 74757, 74761, 76468, 80388]
Updated dataset: 601 records
Removed: 122 records
Done!


National average comparison

2) do the comparison
2) convert bauxite after comparison to aluminium

In [81]:
"""
Simple script to calculate average annual production per mine and aggregate
total annual production by commodity and country.
"""

import pandas as pd
import os

# File paths
INPUT_FILE = "Data Output/Commodity Production/Commodity_Production-1980_2023_final.xlsx"
OUTPUT_DIR = "Data Output/Cumulative Production/Comparison_Results/"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Column names
PROP_ID_COL = 'PROP_ID'
CUMPROD_COL = 'CumProd_Tonnes'
START_YEAR_COL = 'START_UP_YR'
CLOSURE_YEAR_COL = 'PROJ_CLOSURE_YR'
PRIMARY_COMMODITY_COL = 'PRIMARY_COMMODITY'
COUNTRY_COL = 'COUNTRY_NAME'

def main():
    """Calculate annual production and aggregate by commodity and country."""
    
    print("Loading data...")
    df = pd.read_excel(INPUT_FILE, engine='openpyxl')
    
    # Calculate mine lifetime
    df['Mine_Lifetime_Years'] = df[CLOSURE_YEAR_COL] - df[START_YEAR_COL]
    
    # Calculate average annual production
    df['Average_Annual_Production_Tonnes'] = df[CUMPROD_COL] / df['Mine_Lifetime_Years']
    
    print(f"Processed {len(df)} mines")
    
    # Aggregate by commodity and country
    commodity_country = df.groupby([PRIMARY_COMMODITY_COL, COUNTRY_COL]).agg({
        'Average_Annual_Production_Tonnes': 'sum',
        PROP_ID_COL: 'count'
    }).round(2)
    
    commodity_country.columns = ['Total_Annual_Production_Tonnes', 'Number_of_Mines']
    commodity_country = commodity_country.reset_index().sort_values('Total_Annual_Production_Tonnes', ascending=False)
    
    # Save results
    commodity_country.to_excel(os.path.join(OUTPUT_DIR, 'annual_production_by_commodity_country.xlsx'), index=False)
    
    print(f"Results saved to: annual_production_by_commodity_country.xlsx")
    print(f"Top 5 combinations:")
    for i, (_, row) in enumerate(commodity_country.head(5).iterrows()):
        print(f"  {i+1}. {row[PRIMARY_COMMODITY_COL]} in {row[COUNTRY_COL]}: {row['Total_Annual_Production_Tonnes']:,.0f} tonnes/year")

if __name__ == "__main__":
    main()

Loading data...
Processed 609 mines
Results saved to: annual_production_by_commodity_country.xlsx
Top 5 combinations:
  1. Iron Ore in Brazil: 1,167,063,898 tonnes/year
  2. Iron Ore in Australia: 234,531,400 tonnes/year
  3. Iron Ore in Russia: 64,132,909 tonnes/year
  4. Iron Ore in India: 49,734,793 tonnes/year
  5. Iron Ore in China: 42,331,584 tonnes/year


In [106]:
import pandas as pd

# Read the Excel file
file_path = 'Data Output/Commodity Production/Commodity_Production-1980_2023_final.xlsx'
df = pd.read_excel(file_path)

# List of PROP_IDs to remove
prop_ids_to_remove = [37218, 31921, 29665, 28529]

# Remove rows with specified PROP_IDs
df_filtered = df[~df['PROP_ID'].isin(prop_ids_to_remove)]

# Save back to the same file
df_filtered.to_excel(file_path, index=False)

print(f"Removed {len(df) - len(df_filtered)} rows with specified PROP_IDs")
print(f"Remaining rows: {len(df_filtered)}")

Removed 3 rows with specified PROP_IDs
Remaining rows: 598


In [107]:
import pandas as pd

# File paths
file_paths = [
    'Data Output/Cumulative Production/data_records_lifetime.xlsx',
    'Data Output/Cumulative Production/production_model_fits_lifetime.xlsx'
]

# PROP_IDs to remove
prop_ids_to_remove = [
    24470, 24474, 24503, 24505, 24732, 24733, 24774, 24783, 24815, 25687, 25699, 
    25715, 25720, 25727, 25751, 25753, 25754, 25831, 26073, 26495, 26575, 26711, 
    26722, 27027, 27134, 27139, 27145, 27146, 27163, 27165, 27166, 27167, 27213, 
    27243, 27280, 27283, 27291, 27303, 27345, 27384, 27539, 27542, 27543, 27612, 
    27616, 27632, 27653, 27747, 27825, 27830, 27868, 27971, 27981, 28004, 28021, 
    28245, 28250, 28264, 28283, 28296, 28316, 28362, 28363, 28364, 28428, 28585, 
    28596, 28703, 28955, 29057, 29073, 29195, 29246, 29347, 29377, 29438, 29544, 
    29792, 29980, 30277, 30526, 30656, 31300, 31316, 31560, 31563, 31631, 31651, 
    31758, 31818, 31827, 31833, 31895, 31907, 31934, 32142, 32169, 32642, 32779, 
    32931, 35185, 35378, 35755, 35780, 36464, 36896, 37326, 38727, 54759, 56047, 
    57368, 57500, 59345, 64042, 66063, 66682, 66683, 68175, 74744, 74757, 74761, 
    76468, 80388, 37218, 31921, 29665, 28529

]

# Process each file
for file_path in file_paths:
    # Read the Excel file
    df = pd.read_excel(file_path)
    
    # Remove rows with specified PROP_IDs
    df_filtered = df[~df['PROP_ID'].isin(prop_ids_to_remove)]
    
    # Save back to the same file
    df_filtered.to_excel(file_path, index=False)
    
    print(f"{file_path}: Removed {len(df) - len(df_filtered)} rows, {len(df_filtered)} rows remaining")

print("All files updated successfully!")

Data Output/Cumulative Production/data_records_lifetime.xlsx: Removed 13334 rows, 53662 rows remaining
Data Output/Cumulative Production/production_model_fits_lifetime.xlsx: Removed 250 rows, 1196 rows remaining
All files updated successfully!


In [108]:

import pandas as pd

file = "Data Output/Commodity Production/Commodity_Production-1980_2023_final.xlsx"

df = pd.read_excel(file)
mask = df["PRIMARY_COMMODITY"] == "Bauxite"
df.loc[mask, "CumProd_Tonnes"] = df.loc[mask, "CumProd_Tonnes"] * df.loc[mask, "Ore_Grade"]
df.to_excel(file, index=False)

print("Updated CumProd_Tonnes for bauxite mines.")



Updated CumProd_Tonnes for bauxite mines.


Per mine analysis

In [109]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from plotnine import *
from scipy import stats

#########################################Model Functions##############################################

def hubbert_model(t, L, k, t0):
    return L / (1 + np.exp(-k * (t - t0)))

def femp(t, R0, C):
    return R0 * (1 - (1 - C)**t)

def femp_deriv(t, R0, C):
    return - R0 * np.log(1 - C) * (1 - C)**t

def hubbert_deriv(t, L, k, t0):
    exp_term = np.exp(-k * (t - t0))
    return L * k * exp_term / (1 + exp_term)**2

#########################################Utility Functions############################################

def save_fig_plotnine(plot, filename, w=10, h=6):
    plot.save(filename, width=w, height=h, dpi=300)

def save_fig(filename, dpi=300):
    plt.savefig(filename, dpi=dpi, bbox_inches='tight')

#########################################Data Prep#####################################################

def prep_data():
    rec = pd.read_excel('Data Output/Cumulative Production/data_records_lifetime.xlsx')
    modelres = pd.read_excel('Data Output/Cumulative Production/production_model_fits_lifetime.xlsx')
    ua = pd.read_excel('Data Output/Cumulative Production/cumprod_mc_confidence.xlsx')

    rec = rec.rename(columns={'PROP_ID': 'Prop_id'})
    modelres = modelres.rename(columns={'PROP_ID': 'Prop_id'})
    ua = ua.rename(columns={'PROP_ID': 'Prop_id', 'Time_period': 'Year_diff'})

    ua['Year'] = ua['START_UP_YR'] + ua['Year_diff']
    ua['Target_var'] = 'Production'

    rec['Year'] = rec['Year'].astype(int)

    merged = ua.merge(rec[['Prop_id', 'Year', 'Observed']], on=['Prop_id', 'Year'], how='left')
    merged = merged.merge(modelres[['Prop_id', 'Target_var', 'Model', 'R2', 'NRMSE']], on=['Prop_id', 'Target_var', 'Model'], how='left')

    return merged

#########################################Main Plot Function###########################################

def plot_cumprod():
    """
    Build cumulative production curves directly from the model fits:
      - Read 'production_model_fits_lifetime.xlsx'
      - Use P1_value, P2_value, P3_value and Mine_Lifetime
      - For hubbert:  L=P1, k=P2, t0=P3  -> hubbert_model(t, L, k, t0)
      - For femp:     R0=P1, C=P2        -> femp(t, R0, C)
      - Year axis uses START_UP_YR if available; otherwise uses t (years since start)
      - Values converted to Mt
    """
    fits_path = 'Data Output/Cumulative Production/production_model_fits_lifetime.xlsx'
    df = pd.read_excel(fits_path)

    # Standardize column names we rely on
    df = df.rename(columns={'PROP_ID': 'Prop_id'})
    # Keep only the production target if multiple targets exist
    if 'Target_var' in df.columns:
        df = df[df['Target_var'].str.lower() == 'production']

    # Required columns check (be permissive if some exist)
    required = ['Prop_id', 'Model', 'P1_value', 'P2_value', 'Mine_Lifetime']
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns in model fits file: {missing}")

    # If t0 exists for hubbert
    has_p3 = 'P3_value' in df.columns
    has_start = 'START_UP_YR' in df.columns

    # Compute cumulative curves row-wise
    records = []
    for _, row in df.iterrows():
        prop = row['Prop_id']
        model = str(row['Model']).strip().lower()
        p1 = row['P1_value']
        p2 = row['P2_value']
        p3 = row['P3_value'] if has_p3 and not pd.isna(row.get('P3_value')) else None
        lifetime = int(row['Mine_Lifetime']) if not pd.isna(row['Mine_Lifetime']) else 0
        if lifetime < 0:
            continue

        # Build time grid t = 0..lifetime (inclusive)
        t = np.arange(0, lifetime + 1, dtype=float)

        # Compute cumulative according to model
        if model == 'hubbert':
            # Assumed mapping: L=p1, k=p2, t0=p3 (if p3 missing, default to lifetime/2)
            L = p1
            k = p2
            t0 = p3 if p3 is not None else lifetime / 2.0
            cum = hubbert_model(t, L, k, t0)
        elif model == 'femp':
            # Assumed mapping: R0=p1, C=p2
            R0 = p1
            C = p2
            cum = femp(t, R0, C)
        else:
            # Unknown model: skip
            continue

        # Convert to Mt
        cum_mt = cum / 1e6

        # Construct Year or t
        if has_start and not pd.isna(row['START_UP_YR']):
            years = (int(row['START_UP_YR']) + t).astype(int)
            x_name = 'Year'
        else:
            years = t.astype(int)
            x_name = 't'

        # Store
        for xi, yi in zip(years, cum_mt):
            records.append({
                'Prop_id': prop,
                'Model': model,
                x_name: int(xi),
                'CumProd_Mt': yi
            })

    if not records:
        raise ValueError("No cumulative records were generated — check model names/parameters.")

    plot_df = pd.DataFrame(records)

    # Choose x-axis depending on what's available
    if 'Year' in plot_df.columns:
        x_col = 'Year'
        x_lab = 'Year'
    else:
        x_col = 't'
        x_lab = 'Years since start'

    # Sample up to 20 properties to keep the facet grid readable
    sample_ids = plot_df['Prop_id'].dropna().unique()
    if len(sample_ids) > 20:
        sample_ids = np.random.choice(sample_ids, size=20, replace=False)
    plot_df = plot_df[plot_df['Prop_id'].isin(sample_ids)]

    plot = (
        ggplot(plot_df, aes(x=x_col, y='CumProd_Mt', color='Model')) +
        geom_line(size=1) +
        facet_wrap('~Prop_id', scales='free') +
        labs(
            title='Cumulative Production from Model Fits',
            x=x_lab,
            y='Cumulative production (Mt)'
        ) +
        theme_minimal() +
        theme(
            legend_position='bottom',
            panel_background=element_rect(fill='white'),
            plot_background=element_rect(fill='white'),
            strip_background=element_rect(fill='white')
        )
    )

    save_fig_plotnine(
        plot,
        'Visualizations/Cumulative Production/cumprod_plot_from_model_fits.png',
        w=16,
        h=10
    )

def plot_predicted_vs_observed():
    data = pd.read_excel("Data Output/Cumulative Production/data_records_lifetime.xlsx")
    data = data.rename(columns={'PROP_ID': 'Prop_id'})
    data['Observed'] = data['Observed'] / 1e6
    data['Predicted'] = data['Predicted'] / 1e6

    sample_ids = np.random.choice(data['Prop_id'].unique(), size=min(20, data['Prop_id'].nunique()), replace=False)
    subset = data[data['Prop_id'].isin(sample_ids)]

    plot = (
        ggplot(subset, aes(x='Year')) +
        geom_point(aes(y='Observed'), color='black', size=1.5) +
        geom_line(aes(y='Predicted', color='Model'), size=1) +
        facet_wrap('~Prop_id', scales='free') +
        labs(title='Observed vs Predicted Production', x='Year', y='Production (Mt)') +
        theme_minimal() +
        theme(
            legend_position='bottom',
            panel_background=element_rect(fill='white'),
            plot_background=element_rect(fill='white'),
            strip_background=element_rect(fill='white'),
            panel_grid_major=element_line(color='lightgray', size=0.5),
            panel_grid_minor=element_line(color='lightgray', size=0.25)
        )
    )

    save_fig_plotnine(plot, 'Visualizations/Cumulative Production/prod_obs_vs_pred.png', w=16, h=10)

def plot_residuals_by_model():
    data = pd.read_excel("Data Output/Cumulative Production/data_records_lifetime.xlsx")
    data = data.rename(columns={'PROP_ID': 'Prop_id'})
    data['Residual'] = data['Residual'] / 1e6  # Convert to Mt if needed

    # Get common sample IDs that exist for both models
    hubbert_data = data[data['Model'] == 'hubbert']
    femp_data = data[data['Model'] == 'femp']
    
    # Find intersection of property IDs that exist in both models
    common_prop_ids = set(hubbert_data['Prop_id'].unique()) & set(femp_data['Prop_id'].unique())
    common_prop_ids = list(common_prop_ids)
    
    # Sample from the common property IDs
    if len(common_prop_ids) > 20:
        sample_ids = np.random.choice(common_prop_ids, size=20, replace=False)
    else:
        sample_ids = common_prop_ids

    for model in ['hubbert', 'femp']:
        model_data = data[data['Model'] == model]
        subset = model_data[model_data['Prop_id'].isin(sample_ids)]

        plot = (
            ggplot(subset, aes(x='Year', y='Residual')) +
            geom_point(alpha=0.7) +
            geom_smooth(color='red', method='lm') +
            facet_wrap('~Prop_id', scales='free') +
            labs(title=f'Residuals Over Time ({model} Model)', x='Year', y='Residual (Mt)') +
            theme_minimal() +
            theme(
                legend_position='none',
                panel_background=element_rect(fill='white'),
                plot_background=element_rect(fill='white'),
                strip_background=element_rect(fill='white'),
                panel_grid_major=element_line(color='lightgray', size=0.5),
                panel_grid_minor=element_line(color='lightgray', size=0.25)
            )
        )

        filename = f'Visualizations/Cumulative Production/residuals_plot_{model}.png'
        save_fig_plotnine(plot, filename, w=16, h=10)

def qqplot():
    data = pd.read_excel("Data Output/Cumulative Production/data_records_lifetime.xlsx")
    data = data.rename(columns={'PROP_ID': 'Prop_id'})
    targets = ['Observed', 'Predicted']
    subset = data[targets].apply(np.log)

    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    for i, target in enumerate(targets):
        stats.probplot(subset[target].dropna(), dist="norm", plot=ax[i])
        ax[i].set_title(f'QQ Plot - {target}')

    plt.tight_layout()
    save_fig('Visualizations/Cumulative Production/qq_plots.png')
    plt.close()

def normality_tests():
    data = pd.read_excel("Data Output/Cumulative Production/data_records_lifetime.xlsx")
    data = data.rename(columns={'PROP_ID': 'Prop_id'})
    targets = ['Observed', 'Predicted']
    grouped = data.groupby('Prop_id')

    results = []
    for prop_id, group in grouped:
        for var in targets:
            values = group[var].dropna()
            if len(values) < 10:
                continue
            shapiro_p = stats.shapiro(values)[1]
            ks_p = stats.kstest(values, 'norm')[1]
            results.append((prop_id, var, shapiro_p, ks_p))

    df = pd.DataFrame(results, columns=['Prop_id', 'Variable', 'Shapiro_p', 'KS_p'])
    df.to_excel('Data Output/Cumulative Production/normality_test_results.xlsx', index=False)

#########################################Run###########################################################

if __name__ == '__main__':
    plot_cumprod()
    plot_predicted_vs_observed()
    plot_residuals_by_model()
    qqplot()
    normality_tests()


c:\Users\leshe\AppData\Local\Programs\Python\Python312\Lib\site-packages\plotnine\ggplot.py:615: PlotnineWarning: Saving 16 x 10 in image.
c:\Users\leshe\AppData\Local\Programs\Python\Python312\Lib\site-packages\plotnine\ggplot.py:616: PlotnineWarning: Filename: Visualizations/Cumulative Production/cumprod_plot_from_model_fits.png
c:\Users\leshe\AppData\Local\Programs\Python\Python312\Lib\site-packages\plotnine\ggplot.py:615: PlotnineWarning: Saving 16 x 10 in image.
c:\Users\leshe\AppData\Local\Programs\Python\Python312\Lib\site-packages\plotnine\ggplot.py:616: PlotnineWarning: Filename: Visualizations/Cumulative Production/prod_obs_vs_pred.png
c:\Users\leshe\AppData\Local\Programs\Python\Python312\Lib\site-packages\plotnine\layer.py:364: PlotnineWarning: geom_point : Removed 1104 rows containing missing values.
c:\Users\leshe\AppData\Local\Programs\Python\Python312\Lib\site-packages\plotnine\ggplot.py:615: PlotnineWarning: Saving 16 x 10 in image.
c:\Users\leshe\AppData\Local\Program